In [ ]:
import numpy as np
def to_float_with_nan(x):
    try:
        return float(x)
    except ValueError:
        return -1.0

vectorized_to_float_with_nan = np.vectorize(to_float_with_nan)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image,UnidentifiedImageError
import os
import glob
import torchvision.transforms as transforms
import torch
# Mock function to simulate loading data from files
def load_images_for_date(date, image_folder):
    date_str = date.strftime('%Y%m%d')
    images = []
    for hour in range(24):
        pattern=os.path.join(image_folder,date_str,f"???-{date_str}{hour:02d}00.jpg")
        matched_files=glob.glob(pattern)
        if matched_files:
            img_path=matched_files[0]
            try:
                with Image.open(img_path) as img:
                    images.append(img.copy())
            except(OSError,UnidentifiedImageError):
                images.append(None)
        else:
            images.append(None)
    return images

def resize_image(images):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 directly
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    features=[]
    for img in images:
        if img is not None:
            img=transform(img)
            features.append(img)
        else:
            features.append(torch.zeros(3,224,224))
    return features

def load_data_for_segment(station_name, start_date, end_date):
    print(f'Loading data for {station_name} from {start_date} to {end_date}')
    num_data=pd.read_csv(f'./dataset/{station_name}.csv')
    #process the numerical data
    num_data=num_data.drop(columns=['Station'])
    num_data['date']=pd.to_datetime(num_data['date'],format="%d-%m-%Y %H:%M",dayfirst=True)
    #delete all entries that are not in the segment
    print(num_data.head(50))
    num_data=num_data[(num_data['date']>=start_date) & (num_data['date']<end_date)]
    print("Loaded num data!!!")
    #preprocess the data appropriately
    img_data={date:load_images_for_date(date,f'./dataset/{station_name}') for date in num_data['date'].dt.date.unique()}
    print("Loaded image data!!!")
    #process the image data, ie. apply transformation
    image_features = {date: resize_image(images) for date, images in img_data.items()}
    print("Resized image data!!!")
    del img_data
    #reshape the num data
    num_data = num_data.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
    num_data['hour'] = num_data['hour'].astype(int)
    num_data=num_data.pivot(index=['date','hour'],columns='measurement',values='value').reset_index()
    # num_data['date']=num_data['date'].dt.month
    num_data['month']=num_data['date'].dt.month
    print("Processed num data!!!")
    #generate combined features
    combined_features = []
    targets = []
    #instead of each row, we take 18 rows at a time
    for idx, row in num_data.iterrows():
        # print(row)
        date = row['date'].date()
        # print(date)
        hour = row['hour']
        numerical_features = row.drop(['date']).values
        # print(hour)
        # print(numerical_features)
        # print(image_features[date][23])
        img_features = image_features[date][hour]
        # print(img_features.shape)
        #flatten this thing
        img_features = img_features.view(-1)
        #convert to numpy array
        img_features = img_features.detach().numpy()
        print(img_features.shape)
        img_features = img_features.reshape(-1)
        print(numerical_features.shape)
        numerical_features=numerical_features.reshape(-1)
        # print(img_features)
        # print(numerical_features)
        #we can consider adding latitute and longitude as well to the input features
        # numerical_features=np.array(numerical_features).reshape(1,-1)
        # combined_feature=
        # combined_feature=np.array()
        combined_feature = np.concatenate((numerical_features, img_features),axis=None)
        combined_features.append(combined_feature)
        # targets.append(row['value'])  # Assuming the target is the value for that hour
        targets.append(row.drop(['date','hour','month']).values)  # Assuming the target is the value for that hour
    print("Generated combined features and targets!!!")
    combined_features = np.array(combined_features)
    targets = np.array(targets)
    # print(combined_features)
    # print(targets)
    return combined_features, targets

class LargeDatasetSegmentLoader(Dataset):
    def __init__(self, station_ids, segment_duration):
        self.station_ids = station_ids
        self.segment_duration = segment_duration
        self.current_station_index = 0
        self.current_segment_start = None
        self.current_segment_end = None
        self.features = None
        self.targets = None
        self.load_next_segment()

    def load_next_segment(self):
        if self.current_station_index >= len(self.station_ids):
            self.features = None
            self.targets = None
            return
        
        station_id = self.station_ids[self.current_station_index]
        if self.current_segment_start is None:
            self.current_segment_start = pd.to_datetime('2022-01-01')
        
        self.current_segment_end = self.current_segment_start + self.segment_duration
        if self.current_segment_end > pd.to_datetime('2022-12-31'):
            self.current_segment_end = pd.to_datetime('2022-12-31')
        
        self.features, self.targets = load_data_for_segment(
            station_id, self.current_segment_start, self.current_segment_end
        )
        
        self.current_segment_start = self.current_segment_end
        if self.current_segment_start >= pd.to_datetime('2022-12-31'):
            self.current_station_index += 1
            self.current_segment_start = None
    
    def __len__(self):
        return len(self.features) if self.features is not None else 0

    def __getitem__(self, idx):
        if idx >= len(self.features):
            self.load_next_segment()
            idx = 0  # Reset index after loading new segment
        self.features[idx] = vectorized_to_float_with_nan(self.features[idx])
        self.targets[idx] = vectorized_to_float_with_nan(self.targets[idx])
        self.features[idx]=self.features[idx].astype(np.float32)
        self.targets[idx]=self.targets[idx].astype(np.float32)
        print(self.features[idx].shape)
        print(self.features[idx])
        for e in self.features[idx]:
            print(e)
        #some of the values are not floating point, so we need to convert them to float
        feature = torch.tensor(self.features[idx],dtype=torch.float32)
        target = torch.tensor(self.targets[idx],dtype=torch.float32)
        return feature, target
        # Separate numerical and image features
        num_features = torch.tensor(feature[:20], dtype=torch.float32).unsqueeze(1)  # First 20 features
        img_features = torch.tensor(feature[20:], dtype=torch.float32).reshape(3, 224, 224)  # Rest are image features
        target = torch.tensor(target, dtype=torch.float32)
        
        return num_features, img_features, target


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image, UnidentifiedImageError
import os
import glob
import torchvision.transforms as transforms

def load_images_for_date(date, image_folder):
    date_str = date.strftime('%Y%m%d')
    images = []
    for hour in range(24):
        pattern = os.path.join(image_folder, date_str, f"???-{date_str}{hour:02d}00.jpg")
        matched_files = glob.glob(pattern)
        if matched_files:
            img_path = matched_files[0]
            try:
                with Image.open(img_path) as img:
                    images.append(img.copy())
            except (OSError, UnidentifiedImageError):
                images.append(None)
        else:
            images.append(None)
    return images

def resize_image(images):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 directly
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    features = []
    for img in images:
        if img is not None:
            img = transform(img)
            features.append(img)
        else:
            features.append(torch.zeros(3, 224, 224))
    return features

def load_data_for_segment(station_name, start_date, end_date):
    print(f'Loading data for {station_name} from {start_date} to {end_date}')
    num_data = pd.read_csv(f'./dataset/{station_name}.csv')
    num_data = num_data.drop(columns=['Station'])
    num_data['date'] = pd.to_datetime(num_data['date'], format="%d-%m-%Y %H:%M", dayfirst=True)
    # print(num_data.head(50))
    num_data = num_data[(num_data['date'] >= start_date) & (num_data['date'] < end_date)]
    print("Loaded num data!!!")
    img_data = {date: load_images_for_date(date, f'./dataset/{station_name}') for date in num_data['date'].dt.date.unique()}
    print("Loaded image data!!!")
    image_features = {date: resize_image(images) for date, images in img_data.items()}
    print("Resized image data!!!")
    del img_data
    num_data = num_data.melt(id_vars=['date', 'measurement'], var_name='hour', value_name='value')
    num_data['hour'] = num_data['hour'].astype(int)
    num_data = num_data.pivot(index=['date', 'hour'], columns='measurement', values='value').reset_index()
    num_data['month'] = num_data['date'].dt.month
    print("Processed num data!!!")
    combined_features = []
    targets = []

    for idx, row in num_data.iterrows():
        date = row['date'].date()
        hour = row['hour']
        numerical_features = row.drop(['date']).values

        # Convert numerical features to float and handle non-numeric values
        numerical_features = pd.to_numeric(numerical_features, errors='coerce').astype(float)
        
        img_features = image_features[date][hour]
        img_features = img_features.view(-1)
        img_features = img_features.numpy()

        combined_feature = np.concatenate((numerical_features, img_features), axis=None)
        combined_features.append(combined_feature)
        targets.append(row.drop(['date', 'hour', 'month']).values)
        
    print("Generated combined features and targets!!!")
    combined_features = np.array(combined_features)
    targets = np.array(targets)
    combined_feature=vectorized_to_float_with_nan(combined_features)
    targets=vectorized_to_float_with_nan(targets)
    
    return combined_features, targets

class LargeDatasetSegmentLoader(Dataset):
    def __init__(self, station_ids, segment_duration):
        self.station_ids = station_ids
        self.segment_duration = segment_duration
        self.current_station_index = 0
        self.current_segment_start = None
        self.current_segment_end = None
        self.features = None
        self.targets = None
        self.load_next_segment()

    def load_next_segment(self):
        if self.current_station_index >= len(self.station_ids):
            self.features = None
            self.targets = None
            return
        
        station_id = self.station_ids[self.current_station_index]
        if self.current_segment_start is None:
            self.current_segment_start = pd.to_datetime('2022-01-01')
        
        self.current_segment_end = self.current_segment_start + self.segment_duration
        if self.current_segment_end > pd.to_datetime('2022-12-31'):
            self.current_segment_end = pd.to_datetime('2022-12-31')
        
        self.features, self.targets = load_data_for_segment(
            station_id, self.current_segment_start, self.current_segment_end
        )
        
        self.current_segment_start = self.current_segment_end
        if self.current_segment_start >= pd.to_datetime('2022-12-31'):
            self.current_station_index += 1
            self.current_segment_start = None
    
    def __len__(self):
        return len(self.features) if self.features is not None else 0

    def __getitem__(self, idx):
        if idx >= len(self.features):
            self.load_next_segment()
            idx = 0  # Reset index after loading new segment

        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)
        return feature, target

# Example usage
station_ids = ['Keelung', 'Chiayi', 'Tucheng','Banqiao']  # Replace with actual station IDs
segment_duration = pd.DateOffset(months=2)

dataset = LargeDatasetSegmentLoader(station_ids, segment_duration)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example training loop
for epoch in range(200):
    print(f'Epoch {epoch+1}')
    model.train()
    running_loss = 0.0
    for combined_features, targets in dataloader:
        print(combined_features.shape)
        combined_features=combined_features.numpy()
        targets=targets.numpy()
        combined_features=vectorized_to_float_with_nan(combined_features)
        targets=vectorized_to_float_with_nan(targets)
        combined_features=torch.tensor(combined_features,dtype=torch.float32)
        targets=torch.tensor(targets,dtype=torch.float32)
        combined_features, targets = combined_features.to(device), targets.to(device)
        x_num=combined_features[:,:20].unsqueeze(1)
        x_img=combined_features[:,20:]
        x_img=x_img.view(-1,3,224,224)
        #force any non numeric values to be numeric
        
        optimizer.zero_grad()
        outputs = model(x_num, x_img)
        loss = criterion(outputs, targets)
        if torch.isnan(loss):
            print("NaN loss detected")
            print("Inputs: ", combined_features)
            print("Outputs: ", outputs)
            print("Targets: ", targets)
            break
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define the multimodal neural network
class MultimodalNet(nn.Module):
    def __init__(self):
        super(MultimodalNet, self).__init__()
        
        # Define GRU for numerical features
        self.gru_num = nn.GRU(20, 64, 1, batch_first=True)
        
        # Load pre-trained MobileNetV3 as feature extractor
        mobilenet = models.mobilenet_v3_small(pretrained=True)
        self.mobilenet_features = mobilenet.features
        self.mobilenet_classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(576, 128)
        )  # Change to output 128-dim features
        
        # Define linear layers for numerical features
        self.fc1_num = nn.Linear(64, 128)
        self.fc2_num = nn.Linear(128, 64)
        
        # Define linear layers for combined features
        self.fc1_combined = nn.Linear(192, 64)
        self.fc2_combined = nn.Linear(64, 18)  # Output size (18 for regression)

    def forward(self, x_num, x_img):
        # Extract features using GRU for numerical features
        x_num, _ = self.gru_num(x_num)
        x_num = x_num[:, -1, :]  # Only take the last hidden state
        
        # Extract features using MobileNetV3 for image features
        x_img = self.mobilenet_features(x_img)
        x_img = self.mobilenet_classifier(x_img)
        
        # Apply linear layers for numerical features
        x_num = torch.relu(self.fc1_num(x_num))
        x_num = torch.relu(self.fc2_num(x_num))
        
        # Apply linear layers for combined features
        x_combined = torch.cat((x_num, x_img), dim=1)
        x_combined = torch.relu(self.fc1_combined(x_combined))
        x_combined = self.fc2_combined(x_combined)
        
        return x_combined

# Instantiate and train the model
model = MultimodalNet().to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam([
    {'params': model.gru_num.parameters()},  # GRU parameters
    {'params': model.mobilenet_features.parameters(), 'lr': 0.0001},  # MobileNetV3 feature extractor parameters with a lower learning rate
    {'params': model.mobilenet_classifier.parameters(), 'lr': 0.0001},  # MobileNetV3 classifier parameters with a lower learning rate
    {'params': model.fc1_num.parameters()},
    {'params': model.fc2_num.parameters()},
    {'params': model.fc1_combined.parameters()},
    {'params': model.fc2_combined.parameters()}
], lr=0.001)  # Default learning rate for other parameters


In [ ]:
from torch.utils.data import DataLoader, Dataset
# Parameters
station_ids = ['Keelung','Tucheng','Banqiao','Chiayi']
segment_duration = pd.DateOffset(months=2)

# Create dataset and dataloader
dataset = LargeDatasetSegmentLoader(station_ids, segment_duration)
dataloader = DataLoader(dataset, batch_size=32)


In [ ]:
def to_float_with_nan(x):
    try:
        return float(x)
    except ValueError:
        return -1

vectorized_to_float_with_nan = np.vectorize(to_float_with_nan)

In [ ]:
# Training loop
from sklearn.preprocessing import StandardScaler
for epoch in range(20):
    print(f'Epoch {epoch+1}')
    model.train()
    running_loss = 0.0
    for combined_features,targets in dataloader:
        # these combined_features and targets need to be processed
        combined_features = vectorized_to_float_with_nan(combined_features).astype(np.float32)
        targets = vectorized_to_float_with_nan(targets).astype(np.float32)
        # Replace NaNs in combined_features with column means
        print("Replacing NaNs in combined_features with column means...")
        col_mean_combined = np.nanmean(combined_features, axis=0)
        inds_combined = np.where(np.isnan(combined_features))
        combined_features[inds_combined] = np.take(col_mean_combined, inds_combined[1])
        # Replace NaNs in targets with column means
        print("Replacing NaNs in targets with column means...")
        col_mean_targets = np.nanmean(targets, axis=0)
        inds_targets = np.where(np.isnan(targets))
        targets[inds_targets] = np.take(col_mean_targets, inds_targets[1])
        # Normalize the first 20 features
        scaler=StandardScaler()
        combined_features[:,:20]=scaler.fit_transform(combined_features[:,:20])
        # Convert to PyTorch tensors
        combined_features = torch.tensor(combined_features, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)
        # Extract numerical and image features
        num_features = combined_features[:, :20]  # First 20 columns are numerical features
        num_features = num_features.unsqueeze(1)  # Add an extra dimension for GRU
        img_features = combined_features[:, 20:]  # Remaining columns are image features

        # Reshape image features to (batch_size, 3, 224, 224)
        img_features = img_features.reshape(-1, 3, 224, 224)
        #set some of the columns of every row to -1 to simulate missing data
        mask = torch.rand_like(num_features) < 0.1
        num_features[mask] = -1
        # move them to gpu
        num_features, img_features, targets = num_features.to(device), img_features.to(device), targets.to(device)
        # Set some of the values in num_inputs to -1 with a probability of 0.1
        optimizer.zero_grad()
        # outputs = model(num_inputs.unsqueeze(1), img_inputs)
        outputs = model(num_features, img_features)
        loss = criterion(outputs, targets)
        # if torch.isnan(loss):
        #     print("NaN loss detected")
        #     print("Numerical Inputs: ", num_inputs)
        #     print("Image Inputs: ", img_inputs)
        #     print("Outputs: ", outputs)
        #     print("Targets: ", targets_)
        #     break
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss}')
